# Installation

In [ ]:
!pip install sagemaker botocore boto3 awscli --upgrade

In [ ]:
import botocore
import boto3
import sagemaker

print(f"botocore: {botocore.__version__}")
print(f"boto3: {boto3.__version__}")
print(f"sagemaker: {sagemaker.__version__}")

# SageMaker environment

In [ ]:
import sagemaker

sess = sagemaker.Session()

# 기본으로 사용할 버킷 (ex, 학습이 완료된 모델이 업로드)
sagemaker_session_bucket = "{session-bucket-name}" # *변경필요*

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
# sess = sagemaker.Session()

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session_bucket}")
print(f"sagemaker session region: {sess.boto_region_name}")

# Upload datasets

In [ ]:
s3 = boto3.client('s3')

def bucket_exists(bucket_name):
    response = s3.list_buckets()
    for bucket in response['Buckets']:
        print(bucket)
        if bucket['Name'] == bucket_name:
            return True
    return False

# 학습 파일을 업로드할 버킷
bucket_name = 'my-new-bucket-04111' # *변경필요*
if not bucket_exists(bucket_name):
    s3.create_bucket(Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': sess.boto_region_name})

# 업로드할 파일 경로 
train_filename = './data/sample_buy_train.csv'
test_filename = './data/sample_buy_test.csv'
s3.upload_file(train_filename, bucket_name, 'datasets/train/sample_buy_train.csv')
s3.upload_file(test_filename, bucket_name, 'datasets/test/sample_buy_test.csv')

# Train Model

In [ ]:
import os
from sagemaker.sklearn.estimator import SKLearn

train = f"s3://{bucket_name}/datasets/train"
test = f"s3://{bucket_name}/datasets/test"

In [ ]:
sklearn_estimator = SKLearn('sklearn-train-and-deploy.py',
                            role=role,
                            sagemaker_session=sess,
                            instance_type='ml.m4.xlarge',
                            framework_version='1.0-1',
                            hyperparameters = {'epochs': 20, 'batch-size': 64, 'learning-rate': 0.1})
sklearn_estimator.fit({'train': train,
                        'test': test})

In [ ]:
print(sklearn_estimator.latest_training_job.name)
print(sklearn_estimator.model_data)

# Deploy Model

In [ ]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = sklearn_estimator.deploy(instance_type='ml.c4.xlarge',
                                     initial_instance_count=1)

# Deploy Model with training job name

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

# *변경필요*
my_training_job_name = sklearn_estimator.latest_training_job.name
# my_training_job_name = "sagemaker-scikit-learn-2024-01-18-12-17-18-931"
sklearn_estimator = SKLearn.attach(my_training_job_name)
predictor = sklearn_estimator.deploy(instance_type='ml.c4.xlarge',
                                     initial_instance_count=1)

# Deploy Model with existing model url

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

# *변경필요*
model_data = sklearn_estimator.model_data
# model_data = "s3://{model-bucket-name}/sagemaker-scikit-learn-2024-01-18-11-16-30-927/output/model.tar.gz"
sklearn_model = SKLearnModel(model_data=model_data,
                             role=role,
                             entry_point='sklearn-deploy.py',
                             framework_version="1.0-1")

predictor = sklearn_model.deploy(instance_type="ml.c4.xlarge",
                                 initial_instance_count=1)

# Predict Test

In [ ]:
# `data` is a NumPy array or a Python list.
# `response` is a NumPy array.
from sklearn.preprocessing import StandardScaler

x_data = [[80, 8, 7], [60, 8, 9], [25, 3, 2], [19, 4, 5], [30, 3, 3]]
scaler = StandardScaler()
scaler.fit(x_data)
x_norm = scaler.transform(x_data)

response = predictor.predict(x_norm)
print(response)

# Predict Test with boto3

In [ ]:
import boto3
import json
from sklearn.preprocessing import StandardScaler

runtime = boto3.client('runtime.sagemaker')

x_data = [[80, 8, 7], [60, 8, 9], [25, 3, 2], [19, 4, 5], [30, 3, 3]]
scaler = StandardScaler()
scaler.fit(x_data)
x_norm = scaler.transform(x_data)

response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
#     EndpointName='sagemaker-scikit-learn-2024-01-18-11-40-57-770', # 변경필요
    Body=json.dumps(x_norm.tolist()),  
    ContentType='application/json'
)


result = json.loads(response['Body'].read().decode())
print(result)

# Deploy Model with handling request 

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

# *변경필요*
model_data = sklearn_estimator.model_data
# model_data = "s3://{model-bucket-name}/sagemaker-scikit-learn-2024-01-18-12-17-18-931/output/model.tar.gz"
sklearn_model = SKLearnModel(model_data=model_data,
                             role=role,
                             entry_point='sklearn-process-and-deploy.py',
                             framework_version="1.0-1")

predictor = sklearn_model.deploy(instance_type="ml.c4.xlarge",
                                 initial_instance_count=1)

# Predict Test

In [ ]:
# `data` is a NumPy array or a Python list.
# `response` is a NumPy array.
from sklearn.preprocessing import StandardScaler

x_data = [[80, 8, 7], [60, 8, 9], [25, 3, 2], [19, 4, 5], [30, 3, 3]]
response = predictor.predict(x_data)
print(response)

# Predict Test with boto3

In [ ]:
import boto3
import json
from sklearn.preprocessing import StandardScaler

runtime = boto3.client('runtime.sagemaker')

x_data = [[80, 8, 7], [60, 8, 9], [25, 3, 2], [19, 4, 5], [30, 3, 3]]

response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
#     EndpointName='sagemaker-scikit-learn-2024-01-18-12-22-28-194', # 변경필요
    Body=json.dumps(x_data),  
    ContentType='application/json'
)


result = json.loads(response['Body'].read().decode())
print(result)